<a href="https://colab.research.google.com/github/Akinori2/Alocacao_de_laboratorios-SA-/blob/main/Aloca%C3%A7%C3%A3o_de_Laboratorios(SA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bibliotecas usadas  
-random: importa numeros aleatorios   
-math: importa exponenciais e algoritimos   
-deffaultdict: estrutura de dicionário com valores padrão

In [27]:
import random
import math
from collections import defaultdict

Dados das matérias categorizados em graus de importancia, numeros de alunos por materias, se a aula é pratica ou não e o periodo da turma

In [28]:
materias = {
    # Importancia 3
    'Programação 1': {'alunos': 60, 'pratica': True, 'importancia': 3, 'periodo': 1},
    'Estruturas de Dados': {'alunos': 50, 'pratica': True, 'importancia': 3, 'periodo': 3},
    'Banco de Dados': {'alunos': 40, 'pratica': True, 'importancia': 3, 'periodo': 4},
    'Redes de Computadores': {'alunos': 30, 'pratica': True, 'importancia': 3, 'periodo': 5},
    'Inteligência Artificial': {'alunos': 20, 'pratica': True, 'importancia': 3, 'periodo': 6},
    'Desenvolvimento de Software': {'alunos': 30, 'pratica': True, 'importancia': 3, 'periodo': 7},

    # Importancia 2
    'Arquitetura de Computadores': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 2},
    'Sistemas Operacionais': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 3},
    'Engenharia de Software': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 4},
    'Segurança da Informação': {'alunos': 30, 'pratica': True, 'importancia': 2, 'periodo': 5},
    'Computação em Nuvem': {'alunos': 25, 'pratica': True, 'importancia': 2, 'periodo': 6},
    'Computação Gráfica': {'alunos': 20, 'pratica': True, 'importancia': 2, 'periodo': 7},

    # Importancia 1
    'Introdução à Computação': {'alunos': 60, 'pratica': False, 'importancia': 1, 'periodo': 1},
    'Cálculo Numérico': {'alunos': 35, 'pratica': False, 'importancia': 1, 'periodo': 4},
    'Teoria da Computação': {'alunos': 30, 'pratica': False, 'importancia': 1, 'periodo': 5},
    'Sistemas Digitais': {'alunos': 30, 'pratica': True, 'importancia': 1, 'periodo': 6},
    'Ética na Computação': {'alunos': 40, 'pratica': False, 'importancia': 1, 'periodo': 7},
    'Empreendedorismo em TI': {'alunos': 35, 'pratica': False, 'importancia': 1, 'periodo': 8}
}


Salas disponiveis:   
sala G: capacidade alta para alocação de alunos mas equipada com equipamentos mais basicos   
sala P: menor capacidade de alunos mas equipada com melhores equipamentos para tarefas especificas

In [29]:
salas = {
    'Sala Grande': {'limite': 60, 'tipo': 'normal'},
    'Sala Pequena': {'limite': 25, 'tipo': 'especial'}
}

Horarios e dias da semana que a sala poderá atuar

In [30]:
horarios = ['Manhã (8-10)', 'Tarde (10-12)']
dias_semana = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']

Divide as matérias priorizando o grau de importância e quantidade de alunos, logo apos retrona duas grades organizada, facilitando a distribuição nos horários

In [31]:
def separar_materias():
    materias_grande = []
    materias_pequena = []

    for nome, info in materias.items():
        if not info['pratica']:
            continue

        # Prioriza alocar matérias importantes na sala pequena
        if info['alunos'] <= salas['Sala Pequena']['limite']:
            if info['importancia'] >= 2:
                materias_pequena.append(nome)
            elif info['importancia'] == 1 and info['alunos'] > salas['Sala Grande']['limite']:
                materias_pequena.append(nome)
        elif info['alunos'] <= salas['Sala Grande']['limite']:
            materias_grande.append(nome)

    return materias_grande, materias_pequena

Garante que as materias vao preencher toda grade de horario, se necessario completa a grade com outras diciplinas praticas

In [32]:
def completar_horarios(lista_materias, sala):
    total_needed = len(dias_semana) * len(horarios)
    disponiveis = lista_materias.copy()
    # Pega as que se encaixam mesmo não sendo ideais, caso o codigo ache necessario
    if len(disponiveis) < total_needed:
        for nome, info in materias.items():
            if nome not in disponiveis and info['pratica']:
                disponiveis.append(nome)
                if len(disponiveis) >= total_needed:
                    break

    return disponiveis[:total_needed]

Separa as materias e distribui elas aleatoriamente garantindo uma grade completa e equilibrada

In [33]:
def criar_grade_inicial():
    # Separando as matérias
    materias_g, materias_p = separar_materias()
    materias_g = completar_horarios(materias_g, 'Sala Grande')
    materias_p = completar_horarios(materias_p, 'Sala Pequena')
    # Distribuindo aleatoriamente:
    random.shuffle(materias_g)
    random.shuffle(materias_p)

    grade = {}
    for dia in dias_semana:
        for horario in horarios:
            if materias_g:
                grade[('Sala Grande', dia, horario)] = materias_g.pop()
            if materias_p:
                grade[('Sala Pequena', dia, horario)] = materias_p.pop()

    return grade

Avalia a grade e decide o grau de penalidade em um sistema de pontos sendo:  
-salas superlotadas  
-matérias repetidas  
-importância media  

In [34]:
def avaliar_grade(grade):
    pontos = 0
    contagem = defaultdict(int)
    soma_importancia = {'Sala Grande': 0, 'Sala Pequena': 0}
    contagem_salas = {'Sala Grande': 0, 'Sala Pequena': 0}

    for (sala, _, _), materia in grade.items():
        info = materias[materia]
        contagem[materia] += 1
        soma_importancia[sala] += info['importancia']
        contagem_salas[sala] += 1

        # Verifica regras básicas
        if info['alunos'] > salas[sala]['limite']:
            pontos += 30
        if contagem[materia] > 1:
            pontos += 50

    # Calcula as médias
    media_grande = soma_importancia['Sala Grande'] / max(1, contagem_salas['Sala Grande'])
    media_pequena = soma_importancia['Sala Pequena'] / max(1, contagem_salas['Sala Pequena'])

    # Aplica uma penalidade pesada se a média da pequena não for maior
    if media_pequena <= media_grande:
        pontos += 100 * (media_grande - media_pequena + 0.5)

    return pontos

Cria uma variaçao da grade sempre que ocorre uma troca de disciplinas nas grades

In [35]:
def gerar_variacao(grade):
    nova_grade = grade.copy()
    chaves = list(nova_grade.keys())

    if len(chaves) >= 2:
        i, j = random.sample(range(len(chaves)), 2)
        chave1, chave2 = chaves[i], chaves[j]
        nova_grade[chave1], nova_grade[chave2] = nova_grade[chave2], nova_grade[chave1]

    return nova_grade

Implementação do algoritimo SA: \\
Começa com uma grade aleatória e, a cada passo, gera variações trocando disciplinas, aceitando melhorias imediatas ou soluções piores temporariamente

In [36]:
def melhorar_grade():
    melhor = criar_grade_inicial()
    melhor_pontos = avaliar_grade(melhor)
    temp = 1000

    for passo in range(1, 15001):  # Interaçoes podem ser modificadas
        nova = gerar_variacao(melhor)
        pontos = avaliar_grade(nova)

        if pontos < melhor_pontos or random.random() < math.exp((melhor_pontos - pontos)/temp):
            melhor, melhor_pontos = nova, pontos

        temp *= 0.999  # Resfriamento mais lento para melhorar resultados

        if passo % 1000 == 0:
          return melhor

Exibe e formata a grade organizando a mesma por sala, dia e horário

In [37]:
def mostrar_resultado(grade):
    for sala in salas:
        print(f"\n{sala} ({salas[sala]['limite']} alunos)")
        print("-" * 50)
        print(f"{'Dia':<10} | {'8-10h':<20} | {'10-12h':<20}")
        print("-" * 50)

        for dia in dias_semana:
            linha = f"{dia:<10} | "
            for horario in horarios:
                materia = grade.get((sala, dia, horario), "VAZIO")
                info = materias.get(materia, {})

                linha += f"{materia:<20} | "
            print(linha)
        print("-" * 50)

Execução e exibição do codigo

In [38]:
grade_otimizada = melhorar_grade()

print("\nResultado final:")
mostrar_resultado(grade_otimizada)


Resultado final:

Sala Grande (60 alunos)
--------------------------------------------------
Dia        | 8-10h                | 10-12h              
--------------------------------------------------
Segunda    | Inteligência Artificial | Arquitetura de Computadores | 
Terça      | Computação Gráfica   | Sistemas Digitais    | 
Quarta     | Banco de Dados       | Estruturas de Dados  | 
Quinta     | Sistemas Operacionais | Programação 1        | 
Sexta      | Sistemas Operacionais | Desenvolvimento de Software | 
--------------------------------------------------

Sala Pequena (25 alunos)
--------------------------------------------------
Dia        | 8-10h                | 10-12h              
--------------------------------------------------
Segunda    | Redes de Computadores | Banco de Dados       | 
Terça      | Programação 1        | Redes de Computadores | 
Quarta     | Arquitetura de Computadores | Estruturas de Dados  | 
Quinta     | Segurança da Informação | Engenharia de S

Estatísticas das salas e materias  
a preferencia sempre deve ser sobre a maior media da sala pequena em relaçao a sala grande comprovando a priorização correta de disciplinas relevantes no espaço menor

In [39]:
print("\nEstatísticas:")
for sala in salas:
    aulas = [m for (s,_,_), m in grade_otimizada.items() if s == sala]
    print(f"\n{sala}:")
    print(f"- Aulas: {len(aulas)}")
    print(f"- Matérias únicas: {len(set(aulas))}")
    print(f"- Importância média: {sum(materias[m]['importancia'] for m in aulas)/len(aulas):.1f}")


Estatísticas:

Sala Grande:
- Aulas: 10
- Matérias únicas: 9
- Importância média: 2.4

Sala Pequena:
- Aulas: 10
- Matérias únicas: 9
- Importância média: 2.6
